In [50]:
import numpy as np
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [51]:
RAND_STATE = 42

In [52]:
# Mock dataset for demonstration
class TextDataset(Dataset):
    # def __init__(self, vocabulary_size, sequence_length, num_samples):
    #     self.data = torch.randint(0, vocabulary_size, (num_samples, sequence_length))
    #     self.labels = torch.randint(0, 2, (num_samples,))
    def __init__(self, texts, labels, sequence_length):
        self.texts = texts
        self.labels = labels
        self.sequence_length = sequence_length
        self.vocab = self.build_vocab(texts)
        self.encoded_texts = [self.encode_text(text) for text in texts]

    def build_vocab(self, texts):
        unique_words = set(word for text in texts for word in text.lower().split())
        vocab = {word: i + 1 for i, word in enumerate(unique_words)}  # +1 for padding token at index 0
        return vocab
    
    def encode_text(self, text):
        return [self.vocab.get(word, 0) for word in text.lower().split()][:self.sequence_length] + [0] * (self.sequence_length - len(text.split()))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.encoded_texts[idx]), torch.tensor(self.labels[idx])
    

# Define the Q-network model
# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values

# class DQN(nn.Module):
#     def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes):
#         super(DQN, self).__init__()
#         self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
#         self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
#         self.fc = nn.Linear(hidden_dim, num_classes)

#     def forward(self, x):
#         embeds = self.embedding(x)
#         lstm_out, _ = self.lstm(embeds)
#         q_values = self.fc(lstm_out[:, -1])
#         return q_values
    
class DQN(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=0.5, pre_trained_embeddings=None):
        super(DQN, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, embedding_dim)
        if pre_trained_embeddings is not None:
            self.embedding.weight = nn.Parameter(pre_trained_embeddings)
            self.embedding.weight.requires_grad = False  # Or True if you want to fine-tune

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, num_layers=2) # Stacked LSTMs
        self.dropout = nn.Dropout(dropout_rate)  # Use dropout_rate from arguments
        self.fc = nn.Linear(hidden_dim * 2, num_classes)  # Adjust for bidirectional LSTM
        self.relu = nn.ReLU()

    def forward(self, x):
        embeds = self.embedding(x)
        lstm_out, _ = self.lstm(embeds)
        out = self.dropout(lstm_out[:, -1])
        out = self.relu(out)
        q_values = self.fc(out)
        return q_values

In [53]:
def train(model, device, train_loader, optimizer, epoch, log_interval=10):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # Using CrossEntropyLoss which combines LogSoftmax and NLLLoss
        loss = nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def validate(model, device, validation_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            validation_loss += nn.CrossEntropyLoss()(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    validation_loss /= len(validation_loader.dataset)
    validation_acc = correct / len(validation_loader.dataset)
    print(f'\nValidation set: Average loss: {validation_loss:.4f}, Accuracy: {correct}/{len(validation_loader.dataset)} ({100. * correct / len(validation_loader.dataset):.0f}%)\n')
    return validation_loss, validation_acc

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.CrossEntropyLoss()(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [54]:
# Example texts and labels
texts = [
    "Great movie loved it", "Did not like the movie", "Awesome plot and acting",
    "Boring and too long", "Wonderfully made film", "Not my cup of tea",
    "Thrilling and exciting from start to finish", "Could have been better", "A masterpiece", "Waste of time",
    "The director did a fantastic job", "Terrible performances", "The cinematography is gorgeous", 
    "I fell asleep halfway through", "An unforgettable journey", "I've seen better", 
    "A total letdown from a great director", "The soundtrack complemented the movie perfectly", 
    "Lacked depth in the storyline", "Characters were not developed well", "Laughed all the way through", 
    "Failed to deliver a compelling narrative", "Visual effects were stunning", "Plot was predictable", 
    "Highly recommend this movie", "Not worth your time", "An epic disappointment", 
    "Engaging from start to finish", "Script was beautifully written", "The movie felt rushed",
    "A visual masterpiece", "Forgettable experience", "Will watch it again", "Do not waste your money", 
    "The acting was subpar", "A story well told", "Lacks originality", "Captivating to the very end", 
    "Missed the mark on many levels", "One of the year's best films"
]
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1]

# Splitting dataset into train+val and test
train_val_texts, test_texts, train_val_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=RAND_STATE)

# Splitting train+val into train and val
train_texts, validation_texts, train_labels, validation_labels = train_test_split(train_val_texts, train_val_labels, test_size=0.25, random_state=RAND_STATE)  # 0.25 x 0.8 = 0.2

# Creating datasets
sequence_length = 10  # Max number of words in a text
train_dataset = TextDataset(train_texts, train_labels, sequence_length)
validation_dataset = TextDataset(validation_texts, validation_labels, sequence_length)
test_dataset = TextDataset(test_texts, test_labels, sequence_length)

# Creating DataLoaders
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for data, label in train_loader:
    print(f"Train Encoded text: {data}")
    print(f"Train Label: {label}")
    break  # Just show one batch for brevity

for data, label in validation_loader:
    print(f"Validation Encoded text: {data}")
    print(f"Validation Label: {label}")
    break  # Just show one batch for brevity

for data, label in test_loader:
    print(f"Test Encoded text: {data}")
    print(f"Test Label: {label}")
    break  # Just show one batch for brevity

Train Encoded text: tensor([[20, 15,  2, 75,  0,  0,  0,  0,  0,  0],
        [49, 36, 66, 11,  0,  0,  0,  0,  0,  0],
        [42, 10, 62, 47,  1,  5,  0,  0,  0,  0],
        [28,  8, 54, 67,  0,  0,  0,  0,  0,  0]])
Train Label: tensor([0, 0, 0, 1])
Validation Encoded text: tensor([[ 7, 16, 26,  0,  0,  0,  0,  0,  0,  0],
        [11, 22,  1, 30,  0,  0,  0,  0,  0,  0],
        [11,  8,  3, 13,  5, 15,  0,  0,  0,  0],
        [18,  9, 17,  6,  0,  0,  0,  0,  0,  0]])
Validation Label: tensor([1, 0, 1, 1])
Test Encoded text: tensor([[28, 30, 25, 19,  4,  0,  0,  0,  0,  0],
        [12, 10, 23,  9, 12,  1,  8,  0,  0,  0],
        [ 7, 27, 22,  0,  0,  0,  0,  0,  0,  0],
        [ 6,  5, 15,  0,  0,  0,  0,  0,  0,  0]])
Test Label: tensor([0, 0, 0, 0])


In [55]:
# Parameters and Hyperparameters
vocabulary_size = 10000  # to adjust 
sequence_length = 50  # to adjust 
embedding_dim = 128
hidden_dim = 64
num_classes = 2
batch_size = 64
epochs = 10
learning_rate = 0.001

# Model, optimizer, and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    validate(model, device, validation_loader)

# After training, evaluate on the test set
test(model, device, test_loader)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.672709

Validation set: Average loss: 0.2091, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.583727

Validation set: Average loss: 0.2270, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.663663

Validation set: Average loss: 0.2318, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.570075

Validation set: Average loss: 0.2470, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.585030

Validation set: Average loss: 0.2611, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.680431

Validation set: Average loss: 0.2684, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.410269

Validation set: Average loss: 0.2593, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.357329

Validation set: Average loss: 0.2532, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.502893

Validation set: Average loss: 0.2578, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.572427

Validation set: Average loss: 0.2510, 

In [56]:
# Parameters and Hyperparameters
n_trials=50
num_classes = 2

def objective(trial):
    # Define the search space
    vocabulary_size = trial.suggest_categorical('vocabulary_size', [5000, 10000, 20000, 40000])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    embedding_dim = trial.suggest_categorical('embedding_dim', [64, 128, 256])
    hidden_dim = trial.suggest_categorical('hidden_dim', [32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
    step_size = trial.suggest_int('step_size', 1, 100)
    gamma = trial.suggest_float('gamma', 0.1, 1.0, log=True)
    sequence_length = trial.suggest_categorical('sequence_length', [50, 100, 200, 400])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Model setup with trial suggestions
    model = DQN(vocabulary_size, embedding_dim, hidden_dim, num_classes, dropout_rate=dropout_rate).to(device)

    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate)

    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Training loop
    epochs = 10  # Reduced for faster optimization cycles
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        val_loss, val_accuracy = validate(model, device, validation_loader)
        scheduler.step()

    # Set custom attributes for the trial
    trial.set_user_attr("val_loss", val_loss)
    trial.set_user_attr("val_accuracy", val_accuracy)
    
    # print(f"Returning from validate: val_loss={val_loss}, val_accuracy={val_accuracy}")
    # return val_loss

    # Objective: maximize validation accuracy by minimizing its negative value
    return -val_accuracy  # Return the negative accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials)  # Number of trials can be adjusted

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

# Retrieve the validation loss and accuracy from the best trial
best_val_loss = trial.user_attrs["val_loss"]
best_val_accuracy = trial.user_attrs["val_accuracy"]

print(f'Best Validation Loss: {best_val_loss}')
print(f'Best Validation Accuracy: {best_val_accuracy}')
print('Best Trial Parameters:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2024-04-02 19:28:32,524] A new study created in memory with name: no-name-4a1ca502-2b2d-4001-87d4-21922ee6b7b7


Train Epoch: 1 [0/24 (0%)]	Loss: 0.684260

Validation set: Average loss: 0.1173, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.646915

Validation set: Average loss: 0.1250, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.637748

Validation set: Average loss: 0.1300, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.645492

Validation set: Average loss: 0.1229, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.625489

Validation set: Average loss: 0.1306, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.610410

Validation set: Average loss: 0.1348, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.620317

Validation set: Average loss: 0.1256, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.626689

Validation set: Average loss: 0.1184, Accuracy: 1/8 (12%)



[I 2024-04-02 19:28:32,850] Trial 0 finished with value: -0.25 and parameters: {'vocabulary_size': 20000, 'batch_size': 128, 'learning_rate': 0.0002286833928511611, 'embedding_dim': 128, 'hidden_dim': 128, 'optimizer': 'RMSprop', 'dropout_rate': 0.4964945399672808, 'step_size': 35, 'gamma': 0.29737754461183696, 'sequence_length': 200}. Best is trial 0 with value: -0.25.


Train Epoch: 9 [0/24 (0%)]	Loss: 0.602192

Validation set: Average loss: 0.1416, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.563505

Validation set: Average loss: 0.0956, Accuracy: 2/8 (25%)

Train Epoch: 1 [0/24 (0%)]	Loss: 0.693494

Validation set: Average loss: 0.0894, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.681443

Validation set: Average loss: 0.0919, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.674473

Validation set: Average loss: 0.0939, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.665499

Validation set: Average loss: 0.0959, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.663483

Validation set: Average loss: 0.0975, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.662334

Validation set: Average loss: 0.0992, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.657420

Validation set: Average loss: 0.1007, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.655396

Validation set: Average loss: 0.1021, 

[I 2024-04-02 19:28:33,280] Trial 1 finished with value: -0.125 and parameters: {'vocabulary_size': 20000, 'batch_size': 256, 'learning_rate': 2.1977748773294473e-05, 'embedding_dim': 256, 'hidden_dim': 128, 'optimizer': 'RMSprop', 'dropout_rate': 0.24550515389655375, 'step_size': 100, 'gamma': 0.12129374116920898, 'sequence_length': 50}. Best is trial 0 with value: -0.25.


...

Train Epoch: 4 [0/24 (0%)]	Loss: 0.627835

Validation set: Average loss: 0.1340, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.613363

Validation set: Average loss: 0.1405, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.625237

Validation set: Average loss: 0.1410, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.606488

Validation set: Average loss: 0.1373, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.585770

Validation set: Average loss: 0.1298, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.564943

Validation set: Average loss: 0.1236, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.533471

Validation set: Average loss: 0.1183, Accuracy: 1/8 (12%)



[I 2024-04-02 19:28:43,699] Trial 48 finished with value: -0.125 and parameters: {'vocabulary_size': 40000, 'batch_size': 128, 'learning_rate': 0.005756998970849091, 'embedding_dim': 128, 'hidden_dim': 64, 'optimizer': 'SGD', 'dropout_rate': 0.12622719482357236, 'step_size': 29, 'gamma': 0.11271494786235732, 'sequence_length': 400}. Best is trial 10 with value: -0.875.


Train Epoch: 1 [0/24 (0%)]	Loss: 0.694001

Validation set: Average loss: 0.0870, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.689232

Validation set: Average loss: 0.0872, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.689990

Validation set: Average loss: 0.0873, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.692780

Validation set: Average loss: 0.0875, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.690754

Validation set: Average loss: 0.0877, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.689551

Validation set: Average loss: 0.0878, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.693573

Validation set: Average loss: 0.0880, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.691779

Validation set: Average loss: 0.0881, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.684849

Validation set: Average loss: 0.0883, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.688123

Validation set: Average loss: 0.0884, 

[I 2024-04-02 19:28:43,948] Trial 49 finished with value: -0.125 and parameters: {'vocabulary_size': 5000, 'batch_size': 64, 'learning_rate': 0.0006867305423208369, 'embedding_dim': 128, 'hidden_dim': 64, 'optimizer': 'RMSprop', 'dropout_rate': 0.20755394189499718, 'step_size': 53, 'gamma': 0.49626970499249723, 'sequence_length': 100}. Best is trial 10 with value: -0.875.


Train Epoch: 1 [0/24 (0%)]	Loss: 0.688748

Validation set: Average loss: 0.1199, Accuracy: 1/8 (12%)

Train Epoch: 2 [0/24 (0%)]	Loss: 0.643259

Validation set: Average loss: 0.1267, Accuracy: 1/8 (12%)

Train Epoch: 3 [0/24 (0%)]	Loss: 0.635586

Validation set: Average loss: 0.1343, Accuracy: 1/8 (12%)

Train Epoch: 4 [0/24 (0%)]	Loss: 0.649513

Validation set: Average loss: 0.1108, Accuracy: 1/8 (12%)

Train Epoch: 5 [0/24 (0%)]	Loss: 0.643159

Validation set: Average loss: 0.1343, Accuracy: 1/8 (12%)

Train Epoch: 6 [0/24 (0%)]	Loss: 0.612904

Validation set: Average loss: 0.1249, Accuracy: 1/8 (12%)

Train Epoch: 7 [0/24 (0%)]	Loss: 0.581951

Validation set: Average loss: 0.1012, Accuracy: 1/8 (12%)

Train Epoch: 8 [0/24 (0%)]	Loss: 0.565048

Validation set: Average loss: 0.1989, Accuracy: 1/8 (12%)

Train Epoch: 9 [0/24 (0%)]	Loss: 0.575496

Validation set: Average loss: 0.1182, Accuracy: 1/8 (12%)

Train Epoch: 10 [0/24 (0%)]	Loss: 0.414980

Validation set: Average loss: 0.1610, 